## Homework3

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額（`TotalPrice`の合計）を算出し、さらに合計購買金額をベースに降順に並び替えをします。  
3. 合計購買金額の降順に並んだ購買者を$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数。購買者数が$n$で割り切れるとは限らず、この場合の処理は`Pandas.qcut`の処理に準じます。）にグループ分けします。
4. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
5. さらに、それぞれのグループが売上の何割を占めるかを求めてください。全体の売上に対するグループの金額合計値の割合が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とし、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [5]:
# common
import pandas as pd
import numpy as np

# init part(データの読み込みと前処理)
# file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
file_url = "Online Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(
    lambda x: str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5')
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(
    TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)


In [24]:
# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(
    lambda x: str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5')
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(
    TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)


In [61]:
# working place. everything
def homework(target_online_retail_data_tb, n):

    # customerIDごとに購入金額の合計(TotalPrice)を算出
    sumTotalPrice = target_online_retail_data_tb.groupby(
        'CustomerID').TotalPrice.sum().reset_index()

    # sumTOtalPriceを、TotalPriceの降順に並び替え
    sumTotalPrice = sumTotalPrice.sort_values('TotalPrice', ascending=False)

    # sumTotalPriceを5でqcutする
    sumTotalPrice['Group'] = pd.qcut(
        sumTotalPrice.TotalPrice, n, labels=["グループ" + str(i) for i in range(n, 0, -1)])

    # 各Groupごとに、TotalPriceの合計が全体に占める割合を算出
    my_result = sumTotalPrice.groupby('Group').TotalPrice.sum() / \
        sumTotalPrice.TotalPrice.sum()

    # aはSerial
    # aの配列を、項目1 Groupの値で降順に並び替え
    my_result = my_result.sort_values(ascending=False)
    return my_result


In [62]:
homework(target_online_retail_data_tb, 10)


Group
グループ1     0.613751
グループ2     0.132421
グループ3     0.081592
グループ4     0.055334
グループ5     0.038579
グループ6     0.028380
グループ7     0.020183
グループ8     0.014826
グループ9     0.009779
グループ10    0.005156
Name: TotalPrice, dtype: float64

In [57]:
# ["グループ1,"グループ2","グループ3",."グループn"]というリストを作成する
["グループ" + str(i) for i in range(1, 6)]


['グループ1', 'グループ2', 'グループ3', 'グループ4', 'グループ5']

**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).

In [64]:
# 5~1の数字のリストを作成
list(range(5, 0, -1))


[5, 4, 3, 2, 1]